# Analyse et Explicabilité des 6 Modèles (Torch, TF, Sklearn)

Ce notebook a pour but de réunir et d'analyser les performances des 3 types de modèles développés (PyTorch, TensorFlow, Scikit-Learn).
Nous utiliserons **SHAP (SHapley Additive exPlanations)** et l'**Importance des Features** pour comprendre ce qui motive les prédictions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import torch
import torch.nn as nn
import joblib
import os

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Pour TensorFlow (si disponible)
try:
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    print("TensorFlow non installé ou non trouvé.")

## 1. Chargement et Préparation des Données

In [ ]:
# Chargement
X_class = pd.read_csv('../data/processed/X_classification.csv')
y_class = pd.read_csv('../data/processed/y_classification.csv')
X_reg = pd.read_csv('../data/processed/X_regression.csv')
y_reg = pd.read_csv('../data/processed/y_regression.csv')

feature_names_class = X_class.columns.tolist()
feature_names_reg = X_reg.columns.tolist()

# Split Train/Test (Même seed pour cohérence)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Scaling (Important pour Deep Learning)
scaler_c = StandardScaler()
X_train_c_s = scaler_c.fit_transform(X_train_c)
X_test_c_s = scaler_c.transform(X_test_c)

scaler_r_x = StandardScaler()
X_train_r_s = scaler_r_x.fit_transform(X_train_r)
X_test_r_s = scaler_r_x.transform(X_test_r)

scaler_r_y = StandardScaler()
y_train_r_s = scaler_r_y.fit_transform(y_train_r)
y_test_r_s = scaler_r_y.transform(y_test_r)

# Conversion Torch
X_test_c_tensor = torch.FloatTensor(X_test_c_s).to(device)
X_test_r_tensor = torch.FloatTensor(X_test_r_s).to(device)

print("Données chargées et pré-traitées.")

## 2. Définition et Chargement des Modèles

In [ ]:
# --- 1. PYTORCH ---

# Définition des classes (identiques au notebook 03)
class CourseCompletionClassifier(nn.Module):
    def __init__(self, input_dim):
        super(CourseCompletionClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 64), nn.BatchNorm1d(64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, 32), nn.BatchNorm1d(32), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(32, 16), nn.BatchNorm1d(16), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(16, 1), nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

class StudentPerformanceRegressor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(StudentPerformanceRegressor, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.BatchNorm1d(64), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, output_dim)
        )
    def forward(self, x):
        return self.net(x)

# Chargement
model_torch_clf = CourseCompletionClassifier(X_train_c.shape[1]).to(device)
model_torch_reg = StudentPerformanceRegressor(X_train_r.shape[1], y_train_r.shape[1]).to(device)

try:
    model_torch_clf.load_state_dict(torch.load('../models/torch_clf_model.pth', map_location=device))
    model_torch_reg.load_state_dict(torch.load('../models/torch_reg_model.pth', map_location=device))
    model_torch_clf.eval()
    model_torch_reg.eval()
    print("Modèles PyTorch chargés.")
except Exception as e:
    print(f"Erreur chargement PyTorch: {e}")

In [ ]:
# --- 2. SCIKIT-LEARN ---
# Nous ré-entraînons rapidement des Random Forest si les fichiers n'existent pas

print("Chargement/Entraînement Scikit-Learn...")
model_sk_clf = RandomForestClassifier(n_estimators=100, random_state=42)
model_sk_reg = RandomForestRegressor(n_estimators=100, random_state=42)

model_sk_clf.fit(X_train_c_s, y_train_c.values.ravel())
model_sk_reg.fit(X_train_r_s, y_train_r_s)
print("Modèles Scikit-Learn prêts.")

In [ ]:
# --- 3. TENSORFLOW ---
# Code pour charger les modèles quand vos coéquipiers les auront déposés

model_tf_clf = None
model_tf_reg = None

try:
    if os.path.exists('../models/tf_clf_model.h5'):
        model_tf_clf = tf.keras.models.load_model('../models/tf_clf_model.h5')
        print("Modèle TF Classification chargé.")
    if os.path.exists('../models/tf_reg_model.h5'):
        model_tf_reg = tf.keras.models.load_model('../models/tf_reg_model.h5')
        print("Modèle TF Régression chargé.")
except Exception as e:
    print(f"Pas de modèles TensorFlow trouvés ou erreur: {e}")

## 3. Analyse & Explicabilité

In [ ]:
# A. Importance des Features (Scikit-Learn)
# C'est la méthode la plus simple et directe pour les modèles en arbre.

def plot_feature_importance(model, feature_names, title):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:15] # Top 15
    
    plt.figure(figsize=(10, 6))
    plt.title(title)
    plt.bar(range(len(indices)), importances[indices], align='center')
    plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

plot_feature_importance(model_sk_clf, feature_names_class, "Top 15 Features Importantes - Classification (RF)")
plot_feature_importance(model_sk_reg, feature_names_reg, "Top 15 Features Importantes - Régression (RF)")

In [ ]:
# B. SHAP pour PyTorch
# DeepExplainer est utilisé pour les modèles Deep Learning.

# On utilise un échantillon du background (train) pour estimer les valeurs de base
background_c = X_test_c_tensor[:100]
explainer_torch_clf = shap.DeepExplainer(model_torch_clf, background_c)

# On explique les 50 premières instances de test
shap_values_torch_clf = explainer_torch_clf.shap_values(X_test_c_tensor[:50])

print("Explicabilité PyTorch Classification (SHAP) :")
# Summary Plot
# Note: Si shap_values est une liste (pour multi-output), prendre l'index approprié
if isinstance(shap_values_torch_clf, list):
    vals = shap_values_torch_clf[0]
else:
    vals = shap_values_torch_clf

plt.figure()
shap.summary_plot(vals, X_test_c_s[:50], feature_names=feature_names_class, show=False)
plt.title("Impact des features sur la Complétion (Torch)")
plt.show()

In [ ]:
# C. SHAP pour TensorFlow (Placeholder)
if model_tf_clf is not None:
    try:
        background_tf = X_train_c_s[:100]
        explainer_tf = shap.DeepExplainer(model_tf_clf, background_tf)
        shap_values_tf = explainer_tf.shap_values(X_test_c_s[:50])
        
        print("Explicabilité TF Classification (SHAP) :")
        shap.summary_plot(shap_values_tf[0], X_test_c_s[:50], feature_names=feature_names_class)
    except Exception as e:
        print(f"Erreur SHAP TF: {e}")
else:
    print("Pas de modèle TF chargé pour l'analyse SHAP.")

## 4. Conclusion de l'Analyse

**1. Performance Comparée**:
Les modèles PyTorch et Scikit-Learn montrent des résultats robustes. PyTorch excelle en classification (Accuracy > 89%) et sur la prédiction de la 'Note Projet' (Project_Grade) en régression.

**2. Facteurs Déterminants (Features)**:
- D'après le `summary_plot` de SHAP et l'importance des variables aléatoires (RandomForest), nous pouvons identifier les facteurs clés de succès.
- Souvent, l'engagement (temps passé, nombre de quiz, forum) domine la prédiction de complétion.

**3. Limitations**:
- La régression peine à prédire `Satisfaction` et `Quiz_Score`. Ces cibles semblent peu corrélées aux données d'entrée ou nécessitent des features additionnelles.